In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

# Import datasets from roboflow

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

In [ ]:
%cd {HOME}/datasets
!pip install roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="A88aglIXim3OCSF5vNOe")
project = rf.workspace("gabriel-advent-hobto").project("detect-tdz36")
version = project.version(2)
dataset = version.download("yolov8")


# Contoh train code

In [ ]:
!yolo detect train \
    data='/kaggle/working/datasets/detect-2/data.yaml' \
    model=yolov8m.pt \
    epochs=200 \
    imgsz=416 \
    optimizer='Adam' \
    batch=64 \
    lr0=0.0001 \
    dropout=0.2 \
    plots=True \
    save=True

# Train Grid Search

In [ ]:
import subprocess
from itertools import product

param_grid = {
    'model': ['yolov8s.pt'],
    'dropout': [0.2, 0.5],
    'batch': [32, 64],
    'lr': [0.001, 0.0001],
    'optimizer' : ['Adam', 'RMSProp'],
}

# Generate semua kombinasi
param_combinations = list(product(*param_grid.values()))
print(len(param_combinations))

In [ ]:
# Memasukan semua kombinasi parameter ke dalam commands

commands = []
for params in param_combinations:
    model, dropout, batch, lr, optimizer = params
    command = f"yolo detect train \
data='/kaggle/working/datasets/detect-2/data.yaml' \
model='{model}' \
epochs=200 \
imgsz='416' \
optimizer={optimizer} \
lr0={lr} \
batch='{batch}' \
dropout='{dropout}'\
plots=True \
save=True"
    commands.append(command)

In [ ]:
%cd {HOME}

import subprocess

# Loop setiap perintah dan jalankan menggunakan subprocess
for command in commands:
    print(command)
    subprocess.run(command, shell=True)

# Validasi

In [ ]:
%cd {HOME}
model = '/kaggle/working/runs/detect/train/weights/best.pt'
data = '/kaggle/working/datasets/detect-2/data.yaml'

!yolo task=detect mode=val model=model data=data plots=True save_json=True

# Testing Without draw bounding box

In [ ]:
import glob
import pandas as pd
from PIL import Image as PILImage
from IPython.display import display
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
test_path = "test-image"
model = YOLO(f"model-result/model-terbaik/best.pt")

In [ ]:
results = model.predict(source=test_path)
names = model.names

for i, result in enumerate(results):
    boxes = result.boxes
    labels = result.names
    
    detected_labels = []
    
    for box in boxes:
        label = names[int(box.cls)]
        detected_labels.append(label)
        
    print(f"Detected labels for image {i+1}: {detected_labels}")
    res_plotted = result.plot()
    res_plotted_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(res_plotted_rgb)
    plt.axis('off')
    plt.show()


# Testing with draw bounding box

In [ ]:
results = model.predict(source=test_path)
names = model.names
labels = []

# Simpan hasil deteksi di DF
df_results = pd.DataFrame(columns=['Image', 'Labels'])

for r in results:
    # Get bounding box, index kelas, dan skor kepercayaan
    boxes = r.boxes.xyxy
    cls = r.boxes.cls
    conf = r.boxes.conf  # Confidence score
    
    # Mengurutkan bounding box, kelas, dan skor kepercayaan
    sorted_indices = boxes[:, 0].argsort()
    sorted_boxes = boxes[sorted_indices]
    sorted_cls = cls[sorted_indices]
    sorted_conf = conf[sorted_indices]
    
    # Mendapatkan nama kelas
    file_labels = [f"{names[int(c)]} ({conf:.2f})" for c, conf in zip(sorted_cls, sorted_conf)]
    
    labels.append((file_labels, sorted_boxes, sorted_cls, sorted_conf))


In [ ]:
i = 0

image_extensions = ['png', 'jpg', 'jpeg']

for image_extension in image_extensions:
    image_paths = glob.glob(f"{test_path}/*.{image_extension}")
    for image_path in sorted(image_paths):
        print(image_path)
        file_labels, sorted_boxes, sorted_cls, sorted_conf = labels[i]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Membuat bounding box di gambar berdasarkan value bounding box
        for box, cls_idx, conf in zip(sorted_boxes, sorted_cls, sorted_conf):
            x1, y1, x2, y2 = map(int, box)
            label = f"{names[int(cls_idx)]} ({conf:.2f})"
            color = (254, 113, 31)  # Green color for bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)  # Mengurangi ketebalan bounding box menjadi 1
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)  # Mengurangi ketebalan teks menjadi 1
        
        # Menampilkan gambar + bounding box
        display(PILImage.fromarray(img))
        
        # Menampilkan kelas hasil prediksi
        print("Detected Labels:", ", ".join(file_labels))
        print("\n")
        
        # Menambahkan hasil deteksi ke DF
        df_results = pd.concat([df_results, pd.DataFrame({'Image': [image_path], 'Labels': [', '.join(file_labels)]})], ignore_index=True)
        
        i += 1